# 投资者情绪专题
使用所有的数据进行加总

## 数据准备

In [1]:
import sys
import os
import cudf  #CUDA计算
import pandas as pd

# 自行编写的包
sys.path.append('/home/ubuntu/notebook/Investor-Sentiment')
sys.path.append('/usr/local/stata17/utilities')

# Stata
from pystata import config  # noqa

config.init('mp')
# ------------------------------数据集路径----------------------------------#
DATASETS_PATH = '/data/DataSets/investor_sentiment/'


  ___  ____  ____  ____  ____ ©
 /__    /   ____/   /   ____/      17.0
___/   /   /___/   /   /___/       MP—Parallel Edition

 Statistics and Data Science       Copyright 1985-2021 StataCorp LLC
                                   StataCorp
                                   4905 Lakeway Drive
                                   College Station, Texas 77845 USA
                                   800-STATA-PC        https://www.stata.com
                                   979-696-4600        stata@stata.com

Stata license: Single-user 8-core , expiring  1 Jan 2025
Serial number: 501709301094
  Licensed to: Colin's Stata
               Love U

Notes:
      1. Unicode is supported; see help unicode_advice.
      2. More than 2 billion observations are allowed; see help obs_advice.
      3. Maximum number of variables is set to 5,000; see help set_maxvar.


In [2]:
df = cudf.read_parquet(DATASETS_PATH + 'FORUM_SENT_TRANS.parquet')

In [5]:
df.set_index(['PostDate', 'Stockcode']).sort_index().to_pandas()

PostSource  AvgPosterAttention  AvgPosterFans  TotalPosts  \
PostDate Stockcode                                                              
20100113 600166.SH           1                 NaN            NaN           1   
20100114 600166.SH           1                 NaN            NaN           0   
20100115 600166.SH           1                 NaN            NaN           0   
20100116 600166.SH           1                 NaN            NaN           0   
20100117 600166.SH           1                 NaN            NaN           0   
...                        ...                 ...            ...         ...   
20211016 900948.SH           1                 NaN            NaN           0   
         900952.SH           1                 NaN            NaN           0   
         900953.SH           1                 NaN            NaN           0   
         900955.SH           1                 NaN            NaN           0   
         900957.SH           1                 NaN            NaN           0   

                    PositivePosts  NeutralPosts  NegativePosts  AvgWords  \
PostDate Stockcode                                                         
20100113 600166.SH            1.0             0            0.0    1716.0   
20100114 600166.SH            0.0             0            0.0       NaN   
20100115 600166.SH            0.0             0            0.0       NaN   
20100116 600166.SH            0.0             0            0.0       NaN   
20100117 600166.SH            0.0             0            0.0       NaN   
...                           ...           ...            ...       ...   
20211016 900948.SH            0.0             0            0.0       NaN   
         900952.SH            0.0             0            0.0       NaN   
         900953.SH            0.0             0            0.0       NaN   
         900955.SH            0.0             0            0.0       NaN   
         900957.SH            0.0             0            0.0       NaN   

                    AvgReadings  AvgComments  ...  AvgBullUserBarAge  \
PostDate Stockcode                            ...                      
20100113 600166.SH          1.0          0.0  ...                NaN   
20100114 600166.SH          NaN          NaN  ...                NaN   
20100115 600166.SH          NaN          NaN  ...                NaN   
20100116 600166.SH          NaN          NaN  ...                NaN   
20100117 600166.SH          NaN          NaN  ...                NaN   
...                         ...          ...  ...                ...   
20211016 900948.SH          NaN          NaN  ...                NaN   
         900952.SH          NaN          NaN  ...                NaN   
         900953.SH          NaN          NaN  ...                NaN   
         900955.SH          NaN          NaN  ...                NaN   
         900957.SH          NaN          NaN  ...                NaN   

                    AvgBullUserInfluIndex  AvgBullUserPosts  \
PostDate Stockcode                                            
20100113 600166.SH                    NaN               NaN   
20100114 600166.SH                    NaN               NaN   
20100115 600166.SH                    NaN               NaN   
20100116 600166.SH                    NaN               NaN   
20100117 600166.SH                    NaN               NaN   
...                                   ...               ...   
20211016 900948.SH                    NaN               NaN   
         900952.SH                    NaN               NaN   
         900953.SH                    NaN               NaN   
         900955.SH                    NaN               NaN   
         900957.SH                    NaN               NaN   

                    AvgBullUserComments  AvgBearUserBarAge  \
PostDate Stockcode                                           
20100113 600166.SH                  NaN                NaN   
20100114 600166.SH                